In [38]:
# imports
import os
import json
import re
import time
import requests
import pandas as pd
from dotenv import load_dotenv
from tqdm import tqdm

In [39]:
# Config
MODEL = "gpt-4o-mini"
API_URL = "https://openrouter.ai/api/v1/chat/completions"

load_dotenv()
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise RuntimeError("OPENROUTER_API_KEY not set")

HEADERS = {
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
    "Content-Type": "application/json",
}

In [40]:
df = pd.read_csv(
    "data_training_selected_clusters_comments_and_rules.csv",
    usecols=["body", "assigned_rule_cluster"]
)

df["assigned_rule_cluster"] = (
    df["assigned_rule_cluster"].astype(str).str.strip()
)

VALID_LABELS = sorted(df["assigned_rule_cluster"].unique())

In [41]:
def sample_few_shot_examples(
    df,
    n_per_label=2,
    random_state=None
):
    """
    Returns list of (text, label)
    """
    samples = []

    for label, group in df.groupby("assigned_rule_cluster"):
        k = min(n_per_label, len(group))
        sampled = group.sample(k, random_state=random_state)
        for _, row in sampled.iterrows():
            samples.append((row["body"], label))

    return samples

In [42]:
def build_prompt(comment, valid_labels, few_shot_examples=None):
    if few_shot_examples:
        examples = "\n\n".join(
            f"Comment: {text}\nLabel: {label}"
            for text, label in few_shot_examples
        )

        return f"""
Task: Assign exactly ONE label to the comment.

Valid labels:
{chr(10).join(valid_labels)}

Examples:
{examples}

Comment:
{comment}

Output (label only):
""".strip()
    else:
        return f"""
Task: Assign exactly ONE label to the comment.

Valid labels:
{chr(10).join(valid_labels)}

Comment:
{comment}

Output (label only):
""".strip()


In [43]:
def classify_comment(comment, valid_labels, few_shot_examples=None, retries=3):
    prompt = build_prompt(comment, valid_labels, few_shot_examples)

    payload = {
        "model": MODEL,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0,
    }

    for _ in range(retries):
        try:
            r = requests.post(API_URL, headers=HEADERS, json=payload, timeout=30)
            r.raise_for_status()

            output = r.json()["choices"][0]["message"]["content"].strip()
            return output if output in valid_labels else "error"

        except Exception:
            time.sleep(2)

    return "error"


In [44]:
def classify_tuples_from_dataset(
    tuples,
    df,
    mode="zero",
    n_per_label=2,
    random_state=None
):
    """
    tuples: list of (id, text)
    Returns: list of JSON-serializable dicts
    """

    valid_labels = sorted(df["assigned_rule_cluster"].unique())

    few_shot_examples = None
    if mode == "few":
        few_shot_examples = sample_few_shot_examples(
            df,
            n_per_label=n_per_label,
            random_state=random_state
        )

    results = []

    for id_, text in tqdm(tuples):
        label = classify_comment(
            text,
            valid_labels,
            few_shot_examples
        )

        results.append({
            "id": id_,
            "text": text,
            "predicted_cluster": label
        })

    return results


In [45]:
subset = df.sample(10, random_state=42)
tuples = list(zip(subset.index, subset["body"]))

results = classify_tuples_from_dataset(
    tuples,
    df,
    mode="few",
    n_per_label=2,
    random_state=1
)

100%|██████████| 10/10 [00:06<00:00,  1.59it/s]


In [46]:
results

[{'id': 9048,
  'text': 'You are the type of person that makes me hope the collapse comes soon.',
  'predicted_cluster': 'C - 31'},
 {'id': 7178,
  'text': 'I disagree too... I KNOW Niantic said no... but my recent choice is to 3* them. I generally follow every rule they lay down, but this one baffles me. Someone mentioned safety but I don’t think walking on a sidewalk is necessarily any safer than standing next to a pool. People may jump all over me for this, but if my rating tanks then so be it I guess?',
  'predicted_cluster': 'C - 31'},
 {'id': 26965,
  'text': 'Yes YTA but you’re also a monster and unfit to be a parent. You are going to traumatise this child. Children are not toys to be passed around. You will NEVER be this child’s mother when they have a real mother willing and wanting to raise them. Monster!',
  'predicted_cluster': 'C - 31'},
 {'id': 19779,
  'text': 'Ohhh ok I understand that, I will download the video and put it without the channel link\n\nThanks for the warn